# Rotor Boom Length Analysis
This notebook is intended to investigate the expected length of the rotor boom given a certain propeller size and center hub dimensions.

Rotor Boom Properties:

|        Name        |      Properties     |     Description     |
|:------------------:|:-------------------:|:-------------------:|
|  Yield Strength    |      1783 MPa       |     Maximum Stress before Yield    |
|  Youngs Modulus    |       300 GPa       |     Property of the Stiffness      |

Some papers talk about an optimal distance between adjacent motors for improved aerodynamic efficieny.

Within the paper, the length of the arm and distance between adjacent motors is multiplied by a ratio relating to the diameter of the propeller, $D_{propeller}$, installed on the motor. The study explored ratios, $ \sigma = [1.0, 1.2, 1.4, 1.6, 1.8]$, and the effective aerodynamic improvement based on these ratios [1].

$$Arm Length = \sigma \times D_{propeller}$$

Lei [1] however, did not draw any conclusion on a specific function that relates the ratio, $\sigma$, to the aerodynamic performance but within their circumstances a higher ratio did improve the aerodynamic performance of their sUAS(small Unmanned Aircraft System). Thus there is no guarentee that the optimal ratio within Lei's [1] study will be the optimal solution within this study.

### Minimum Rotor Boom Length
The minimum rotor boom length is equal to the propeller diameter. This distance is measured from a point, $O$, with an axis running through it. Point, $O$, acts as the relative origin and is the axis about which the rotor booms are evenly spaced by $60^{\circ}$, in the case of a hexacopter.

This can be shown through the following image proof which is based on an isosceles triangle. For the case of a hexacopter where the rotor boom lengths are equal and the rotor booms are spaced $60^{\circ}$ apart, an equilateral triangle is formed and thus all sides are the same length. 

In [12]:
# Typical Imports
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Visual Aid For the problem definition and Layout

#### Figure 1:
Hexacopter Layout Description:

|        Name        |      Description     |     Value    |
|:------------------|:-------------------:|:-------------------:|
| Wingspan    |     Distance between opposite motor centers       |     900mm   |
| Boom Length    |     CF Boom Length      |     373mm   |
| Propeller Size    |     Diameter of propeller      |     431.8mm   |
| Adjacent Distance    |     Distance between adjacent motor centers       |     470mm   |
| Angle    |     Angle between rotor booms       |     60$^{\circ}$   |

<img src="OblongSizeEstimation.png" alt="drawing" width="650"/>

### CF Boom Performance Data

In [37]:
boom_data = pd.read_csv('clean_CFRotorBoom20Kg.csv')
mounting_space = 20 + 12 # Space required to mount the motor and boom to the chassis, i.e. where the mounting holes are.
boom_data["HoleToHoleLength"] = boom_data['TubeLength'] - mounting_space
boom_data.head()

,TubeLength,TubeOD,Displacement1,Stress1,HoleToHoleLength
0,250.0,10.0,1.05703,124.42,218.0
1,275.0,10.0,1.45581,138.81,243.0
2,300.0,10.0,1.94392,153.12,268.0
3,325.0,10.0,2.53048,164.56,293.0
4,350.0,10.0,3.22465,178.73,318.0


### Propeller Definition

In [34]:
diam = 17 # Propeller Diameter in inches(in)
prop_diam_mm = np.round(diam * 25.4,1) # Propeller Diameter in millimeters(mm)
print("Propeller Diameter:",np.round(prop_diam_mm,1),"mm")
prop_dist_required = prop_diam_mm*1.10 # The min dist is the same as the propeller diameter realistically this should be higher.
print("Min Dist Between Motors:", np.round(prop_dist_required,1),"mm")

Propeller Diameter: 431.8 mm
Min Dist Between Motors: 475.0 mm


### Boom Length Requirements and Constants

In [60]:
sigma = [1.0, 1.2, 1.4, 1.6, 1.8, 2.0] # sigma as described above
chassis_hole_dist = [105, 125, 150, 175, 200]
min_boom_length = np.round(prop_dist_required,1)

### Boom Length Analysis
The following set of code will evaluate the SOLIDWORKS data and apply the following method:
1. Given a chassis hole distance, does the `boom_data["HoleToHoleLength"]` meet the `min_boom_length`? <br>i.e. `boom_data["HoleToHoleLength"]`$\ge$ `min_boom_length`.


2. 
    True: Show the difference greater than<br>
    False: Show the still distance required<br>


3. Calculate the relative ratio of `boom_data["HoleToHoleLength"]` to `min_boom_length`

#### Shortage or Excess of Length compared to `min_boom_length`

In [61]:
boom_data["HoleDiff"] = (boom_data['HoleToHoleLength'] + chassis_hole_dist[0]) - min_boom_length
boom_data.head()

,TubeLength,TubeOD,Displacement1,Stress1,HoleToHoleLength,HoleDiff,AeroRatio
0,250.0,10.0,1.05703,124.42,218.0,-152.0,1.0
1,275.0,10.0,1.45581,138.81,243.0,-127.0,1.0
2,300.0,10.0,1.94392,153.12,268.0,-102.0,1.0
3,325.0,10.0,2.53048,164.56,293.0,-77.0,1.0
4,350.0,10.0,3.22465,178.73,318.0,-52.0,1.0


#### What is the Ratio of Boom Length to `min_boom_length`

In [62]:
def ratio_func(temp_data):
    #print(temp_data)
    if temp_data['HoleDiff'] > 0:
        return np.round((temp_data['HoleToHoleLength']+chassis_hole_dist[0])/min_boom_length,2)
    else:
        return 1.00
    
boom_data['AeroRatio'] = boom_data.apply(ratio_func,axis=1)
boom_data.head()

,TubeLength,TubeOD,Displacement1,Stress1,HoleToHoleLength,HoleDiff,AeroRatio
0,250.0,10.0,1.05703,124.42,218.0,-152.0,1.0
1,275.0,10.0,1.45581,138.81,243.0,-127.0,1.0
2,300.0,10.0,1.94392,153.12,268.0,-102.0,1.0
3,325.0,10.0,2.53048,164.56,293.0,-77.0,1.0
4,350.0,10.0,3.22465,178.73,318.0,-52.0,1.0


In [63]:
print(min_boom_length)
boom_data[boom_data['TubeOD']==15]

475.0


,TubeLength,TubeOD,Displacement1,Stress1,HoleToHoleLength,HoleDiff,AeroRatio
11,250.0,15.0,0.29390,59.486,218.0,-152.0,1.00
12,275.0,15.0,0.40283,65.987,243.0,-127.0,1.00
13,300.0,15.0,0.53588,72.783,268.0,-102.0,1.00
14,325.0,15.0,0.69554,79.485,293.0,-77.0,1.00
15,350.0,15.0,0.88417,86.146,318.0,-52.0,1.00
16,375.0,15.0,1.10412,92.753,343.0,-27.0,1.00
17,400.0,15.0,1.35808,101.810,368.0,-2.0,1.00
18,425.0,15.0,1.64823,108.300,393.0,23.0,1.05
19,450.0,15.0,1.97683,112.960,418.0,48.0,1.10
20,475.0,15.0,2.34683,119.330,443.0,73.0,1.15


## References:
    [1] MDPI and ACS Style Lei, Y.; Huang, Y.; Wang, H. Aerodynamic Performance of an Octorotor SUAV with Different Rotor Spacing in Hover. Processes 2020, 8, 1364. [https://doi.org/10.3390/pr8111364]
   [Link [1]](https://doi.org/10.3390/pr8111364)